<a href="https://colab.research.google.com/github/aruntakhur/SitareUniversity/blob/main/MinMax_AlphaPruning_AI_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TicTacToe with Minimax and Alpha-Beta Pruning

This implementation demonstrates a complete object-oriented Tic-Tac-Toe game using the Minimax algorithm with Alpha-Beta pruning optimization.


## Class Structure and Initialization

### Board Representation
- **9-element list**: Positions 0-8 representing a 3x3 grid
- **Empty spaces**: Represented by `" "` (space character)
- **Players**: `"X"` and `"O"`

## Game State Management

### Core Methods

#### [print_board()](cci:1://file:///c:/Desktop/3rd%20SEM/AI/interactive_tictactoe.py:127:0-142:11)
- **Purpose**: Visual representation of current game state
- **Format**: Traditional 3x3 grid with `|` and `-` separators

#### `player(state=None)`
- **Logic**: Count X's and O's to determine current player
- **Rule**: X goes first, so equal counts = X's turn
- **Return**: `"X"` or `"O"`

#### `actions(state=None)`
- **Purpose**: Get all legal moves (empty positions)
- **Return**: List of indices where moves can be made
- **Example**: `[0, 2, 4, 6]` for positions that are empty

#### `result(state, action)`
- **Purpose**: Generate new state after making a move
- **Process**: Copy current state, place current player's symbol
- **Return**: New board state (doesn't modify original)

## Win Detection and Terminal States

### `check_winner(player, state=None)`
**Win Conditions**:
- **Rows**: [0,1,2], [3,4,5], [6,7,8]
- **Columns**: [0,3,6], [1,4,7], [2,5,8]  
- **Diagonals**: [0,4,8], [2,4,6]

**Algorithm**: Check if player has 3 symbols in any winning line

### `terminal(state=None)`
**Terminal conditions**:
- X wins
- O wins  
- Board is full (draw)

### `utility(state=None)`
**Utility values**:
- **+1**: X wins (good for maximizing player)
- **-1**: O wins (good for minimizing player)
- **0**: Draw or non-terminal state

## AI Decision Making & Game Execution

### `minimax_decision(state=None)`

**Algorithm Flow**:
1. **Reset counter**: `call_count = 0`
2. **Determine player**: X (maximize) or O (minimize)
3. **For each legal action**:
   - Generate resulting state
   - Evaluate using appropriate value function
   - Track best action found
4. **Return**: Best action with performance statistics

### Performance Benefits
- **Without pruning**: Explores all possible game trees
- **With Alpha-Beta**: Eliminates up to 50-90% of nodes
- **Practical impact**: Faster decisions, deeper search possible

### Game Flow
1. **Initialize**: Empty board, X goes first (human)
2. **Game loop**: Alternate between human and AI moves
3. **AI decision**: Use Minimax with Alpha-Beta pruning
4. **Performance tracking**: Display states evaluated
5. **Terminal check**: Detect wins/draws and announce result

In [1]:
class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]
        self.call_count = 0   # count recursive calls for AI

    def print_board(self):
        """Print the board in a 3x3 grid"""
        print(self.board[0]+"|"+self.board[1]+"|"+self.board[2])
        print("-+-+-")
        print(self.board[3]+"|"+self.board[4]+"|"+self.board[5])
        print("-+-+-")
        print(self.board[6]+"|"+self.board[7]+"|"+self.board[8])

    def player(self, state=None):
        """Return which player's turn it is"""
        s = state if state else self.board
        X_count = s.count("X")
        O_count = s.count("O")
        return "X" if X_count == O_count else "O"

    def actions(self, state=None):
        """Return list of legal moves"""
        s = state if state else self.board
        return [i for i in range(9) if s[i] == " "]

    def result(self, state, action):
        """Return state after making a move"""
        new_state = state.copy()
        new_state[action] = self.player(state)
        return new_state

    def check_winner(self, player, state=None):
        """Check if player has won"""
        s = state if state else self.board
        win_conditions = [
            [0,1,2],[3,4,5],[6,7,8], # rows
            [0,3,6],[1,4,7],[2,5,8], # cols
            [0,4,8],[2,4,6]          # diagonals
        ]
        for cond in win_conditions:
            if s[cond[0]] == s[cond[1]] == s[cond[2]] == player:
                return True
        return False

    def terminal(self, state=None):
        """Check if state is terminal (win or draw)"""
        s = state if state else self.board
        return (self.check_winner("X", s) or
                self.check_winner("O", s) or
                " " not in s)

    def utility(self, state=None):
        """Return +1 if X wins, -1 if O wins, 0 otherwise"""
        s = state if state else self.board
        if self.check_winner("X", s): return 1
        elif self.check_winner("O", s): return -1
        else: return 0

    # ---------------- Minimax with Alpha-Beta ---------------- #

    def max_value(self, state, alpha, beta):
        self.call_count += 1
        if self.terminal(state):
            return self.utility(state)
        v = float("-inf")
        for action in self.actions(state):
            v = max(v, self.min_value(self.result(state, action), alpha, beta))
            if v >= beta:   # beta cutoff
                return v
            alpha = max(alpha, v)
        return v

    def min_value(self, state, alpha, beta):
        self.call_count += 1
        if self.terminal(state):
            return self.utility(state)
        v = float("inf")
        for action in self.actions(state):
            v = min(v, self.max_value(self.result(state, action), alpha, beta))
            if v <= alpha:  # alpha cutoff
                return v
            beta = min(beta, v)
        return v

    def minimax_decision(self, state=None):
        """Return best action using Minimax + Alpha-Beta"""
        self.call_count = 0
        s = state if state else self.board
        player = self.player(s)

        if player == "X":   # maximize
            best_val = float("-inf")
            best_action = None
            for action in self.actions(s):
                val = self.min_value(self.result(s, action), float("-inf"), float("inf"))
                if val > best_val:
                    best_val = val
                    best_action = action
        else:               # minimize
            best_val = float("inf")
            best_action = None
            for action in self.actions(s):
                val = self.max_value(self.result(s, action), float("-inf"), float("inf"))
                if val < best_val:
                    best_val = val
                    best_action = action

        print(f"AI evaluated {self.call_count} states before choosing move {best_action+1}")
        return best_action

    # ---------------- Game Play ---------------- #

    def play(self):
        state = self.board
        while not self.terminal(state):
            self.print_board()
            if self.player(state) == "X":   # Human
                move = int(input("Enter your move (1-9): ")) - 1
                if move not in self.actions(state):
                    print("Invalid move! Try again.")
                    continue
            else:                           # AI
                print("AI is thinking...")
                move = self.minimax_decision(state)

            state = self.result(state, move)
            self.board = state  # update board

        # Final result
        self.print_board()
        if self.utility(state) == 1:
            print("X wins!")
        elif self.utility(state) == -1:
            print("O wins!")
        else:
            print("It's a draw!")


In [2]:
game = TicTacToe()
game.play()


 | | 
-+-+-
 | | 
-+-+-
 | | 
 | | 
-+-+-
X| | 
-+-+-
 | | 
AI is thinking...
AI evaluated 7641 states before choosing move 1
O| | 
-+-+-
X| | 
-+-+-
 | | 
O| | 
-+-+-
X| | 
-+-+-
X| | 
AI is thinking...
AI evaluated 552 states before choosing move 2
O|O| 
-+-+-
X| | 
-+-+-
X| | 
O|O| 
-+-+-
X| | 
-+-+-
X| |X
AI is thinking...
AI evaluated 27 states before choosing move 3
O|O|O
-+-+-
X| | 
-+-+-
X| |X
O wins!
